## Finetuning MORPH on [ICF data](https://github.com/LLNL/macc)

In [ ]:
# install required packages to download from Hugging Face Hub
%pip -q install huggingface_hub
%pip install hf_xet

In [ ]:
# install required packages to get notebook path
%pip install ipynbname --quiet

#### Set-up directories

In [ ]:
# Set cwd to the directory 
import os
from pathlib import Path
import ipynbname

HERE = Path(ipynbname.path()).resolve().parent
ROOT = HERE.parents[1]
os.chdir(HERE)
print("Current directory: ", os.getcwd())
os.chdir(ROOT)
print("Root directory:", os.getcwd())

In [ ]:
# Define important directories
dataset_dir = os.path.join(ROOT, "datasets")
data_dir = os.path.join(ROOT, "data")
model_dir = os.path.join(ROOT, "models")
results_dir = os.path.join(ROOT, "experiments", "results")
print("Dataset directory:", dataset_dir)
print("Data directory:", data_dir)
print("Model directory:", model_dir)
print("Results directory:", results_dir)

#### Load ICF dataset

In [ ]:
# Load ICF-JAG-10K dataset
import os, numpy as np

path_images = os.path.join(dataset_dir, "icf-jag-10k", "jag10K_images.npy")
path_params = os.path.join(dataset_dir, "icf-jag-10k", "jag10K_params.npy")
path_scalars = os.path.join(dataset_dir, "icf-jag-10k", "jag10K_0_scalars.npy")

images = np.load(path_images, allow_pickle=False).astype(np.float32)
params = np.load(path_params, allow_pickle=False).astype(np.float32)
scalars = np.load(path_scalars, allow_pickle=False).astype(np.float32)

print("images.shape:", images.shape)
print("params.shape:", params.shape)
print("scalars.shape:", scalars.shape)

#### Data stats

In [ ]:
# some stats
min_images, max_images = images.min(), images.max()
mean_images, std_images = images.mean(), images.std()
print(f"images: min {min_images}, max {max_images}, mean {mean_images}, std {std_images}")

min_params, max_params = params.min(), params.max()
mean_params, std_params = params.mean(), params.std()
print(f"params: min {min_params}, max {max_params}, mean {mean_params}, std {std_params}")

min_scalars, max_scalars = scalars.min(), scalars.max()
mean_scalars, std_scalars = scalars.mean(), scalars.std()
print(f"scalars: min {min_scalars}, max {max_scalars}, mean {mean_scalars}, std {std_scalars}")

#### Data visualizations

##### Scalars and Parameters

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(6, 4))
plt.subplot(1,2,1)
plt.plot(scalars[0:5,:],'-o')
plt.title('Scalars')

plt.subplot(1,2,2)
plt.plot(params[0:5,:],'-*')
plt.title('Parameters')

##### Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# If not already reshaped:
images_reshape = images.reshape(images.shape[0], 64, 64, 4).astype(np.float32)
print(f"Reshaped images: {images_reshape.shape}")  # (N, 64, 64, 4)

# pick 5 unique samples
idxs = np.random.choice(images_reshape.shape[0], 5, replace=False)

fig, axes = plt.subplots(len(idxs), 4, figsize=(8, 2*len(idxs)))

# handle the case axes is 2D
for r, idx in enumerate(idxs):
    for c in range(4):
        ax = axes[r, c]
        ax.imshow(images_reshape[idx, :, :, c], cmap='plasma', origin='lower')
        ax.set_xticks([])
        ax.set_yticks([])
        if r == 0:
            ax.set_title(f'Channel {c}')
        if c == 0:
            ax.set_ylabel(f'Sample {idx}')

plt.suptitle('Five samples × four channels')
plt.tight_layout()
plt.show()

#### Data Normalization
- We perform Reverse Instance Normalization (ReVIN) [Paper](https://openreview.net/forum?id=cGDAkQo1C0p).
- For every (Sample, Field), it calculates ($\mu$,$\sigma$)
- After training, ($\mu$,$\sigma$) can be used for denormalization

##### Images

In [ ]:
# Data in (N, T, D, H, W, C, F) format
E_1 = images_reshape[:, :, :, 0]
E_2 = images_reshape[:, :, :, 1]
E_3 = images_reshape[:, :, :, 2]
E_4 = images_reshape[:, :, :, 3]

# Organize data into components
E_12 = np.stack((E_1, E_2), axis=-1)[..., np.newaxis]  # (N, H, W, C, F) -> (N, 64, 64, 2, 1)
E_34 = np.stack((E_3, E_4), axis=-1)[..., np.newaxis]  # (N, H, W, C, F) -> (N, 64, 64, 2, 1)
E_13 = np.stack((E_1, E_3), axis=-1)[..., np.newaxis]  # (N, 64, 64, 2, 1)
E_24 = np.stack((E_2, E_4), axis=-1)[..., np.newaxis]  # (N, 64, 64, 2, 1)

# Organize data into fields and components
# type - 1 where E12 and E34 are groups as fields 
data_1 = np.concatenate((E_12, E_34), axis=-1)  # (N, 64, 64, 2, 2)
print(f"Organized data shape (type-1): {data_1.shape}")  # (N, 64, 64, 2, 2)

# type - 2 where E13 and E24 are groups as fields
data_2 = np.concatenate((E_13, E_24), axis=-1)  # (N, 64, 64, 2, 2)
print(f"Organized data shape (type-2): {data_2.shape}")  # (N, 64, 64, 2, 2)

data  = data_1  # choose one organization
print(f"Final data shape (N,H,W,C,F): {data.shape}")  # (N, 64, 64, 2, 2)


In [ ]:
# Bring data into (N, T, D, H, W, C, F) format
data = data[:, np.newaxis, np.newaxis, :, :, :, :]  # (N, 1, 1, 64, 64, 2, 2)
print(f"Data shape (N, T, D, H, W, F, C): {data.shape}")  # (N, 1, 1, 64, 64, 2, 2)

# Reshape the data in UPTF-7 format
data = data.transpose(0, 1, 6, 5, 2, 3, 4)  # (N, T, F, C, D, H, W)
print(f"Data shape (N, T, F, C, D, H, W): {data.shape}")  # (N, 1, 2, 2, 1, 64, 64)

In [ ]:
# Call normalization function
from src.utils.normalization import RevIN
rev_icf = RevIN(data_dir)

# compute & normalize -> Needs data in UPTF-7 format
rev_icf.compute_stats(data, prefix='stats_icf')
dataset_icf_norm = rev_icf.normalize(data, prefix='stats_icf')
print("Normalize dataset shape", dataset_icf_norm.shape)

In [ ]:
# Check round‐trip via denormalize
tol_2 = 1e-4
recovered = rev_icf.denormalize(dataset_icf_norm, prefix='stats_icf')
print("Denormalized dataset shape", recovered.shape)

max_error = 0.0
for i in range(recovered.shape[0]):
    maxerror_i = np.max(np.abs(recovered[i] - data[i]))  # saving some memory
    max_error = max(maxerror_i, max_error)
assert max_error < tol_2, "Denormalization did not perfectly recover original!"
print("RevIN round-trip OK")
del recovered

##### Parameters

In [ ]:
# Visualize the parameters
import matplotlib.pyplot as plt
colors = ['C0', 'C1', 'C2', 'C3', 'C4']
plt.figure(figsize=(20, 4))
for i in range(params.shape[1]):
    plt.subplot(1, 5, i+1)
    plt.hist(params[:,i], bins=50, alpha=0.7, color = colors[i])
    plt.title(f'Parameter {i+1}', fontsize=16)
    plt.xlabel('Value', fontsize=16)
    if i == 0:
        plt.ylabel('Frequency', fontsize=16)
    plt.legend([f'Parameter {i+1}'])

In [ ]:
# Normalize function
def normalize_params(params):
    for i in range(params.shape[1]):
        mu_params = params[:,i].mean()
        std_params = params[:,i].std()
        print(f"Param {i+1}: mean {mu_params}, std {std_params}")
        params[:,i] = (params[:,i] - mu_params) / std_params
    return params

# Denormalize function
def denormalize_params(norm_params, mu_params, std_params):
    return norm_params * std_params + mu_params

# Normalize the parameters
params_norm = normalize_params(params.copy())

# Visualize the parameters
import matplotlib.pyplot as plt
colors = ['C0', 'C1', 'C2', 'C3', 'C4']
plt.figure(figsize=(20, 4))
for i in range(params_norm.shape[1]):
    plt.subplot(1, 5, i+1)
    plt.hist(params_norm[:,i], bins=50, alpha=0.7, color = colors[i])
    plt.title(f'Parameter {i+1}', fontsize=16)
    plt.xlabel('Value', fontsize=16)
    if i == 0:
        plt.ylabel('Frequency', fontsize=16)
    plt.legend([f'Parameter {i+1}'])

In [ ]:
# some stats
for i in range(4):
    min_images, max_images = dataset_icf_norm[..., i].min(), dataset_icf_norm[..., i].max()
    print(f"images: min {min_images}, max {max_images}")

for j in range(5):
    min_params, max_params = params_norm[..., j].min(), params_norm[..., j].max()
    print(f"params: min {min_params}, max {max_params}")

#### Set Up Dataloaders

In [ ]:
# data-scarce and compute-scarce scenarios
dataset_size = 1000  # adjust as needed
num_epochs = 10  # adjust as needed
data_idx = np.random.choice(dataset_icf_norm.shape[0], dataset_size, replace=False)

In [ ]:
# Device information
import torch
print("Number of CUDA devices",torch.cuda.device_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# dataset class
class DatasetforDataloader(Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,i):
        # create a tuple
        return self.X[i], self.y[i]
    
# Dataset of dataloader
X = dataset_icf_norm[data_idx]
y = params_norm[data_idx]
full_dataset = DatasetforDataloader(X, y)

# define the splits (80/10/10)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# Split the dataset 
train, val, test = torch.utils.data.random_split(full_dataset,[train_size, val_size, test_size])


In [ ]:
# Dataloaders
BATCH_SIZE = 8
dataloader_train = DataLoader(train, batch_size = BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(val, batch_size = BATCH_SIZE, shuffle=False)
dataloader_test = DataLoader(test, batch_size = BATCH_SIZE, shuffle=False)
print(f"Number of training samples: {len(dataloader_train)}")
print(f"Number of validation samples: {len(dataloader_val)}")
print(f"Number of test samples: {len(dataloader_test)}")

#### Define Models

##### Load MORPH

In [ ]:
# Instantiate the foundational model
from src.utils.vit_conv_xatt_axialatt2 import ViT3DRegression

MORPH_MODELS = {
    'Ti': [8, 256,  4,  4, 1024],
    'S' : [8, 512,  8,  4, 2048],
    'M' : [8, 768, 12,  8, 3072],
    'L' : [8, 1024,16, 16, 4096]
    }

model_size = MORPH_MODELS['Ti']  # choose from 'Ti', 'S', 'M', 'L'
filters, dim, depth, heads, mlp_dim = model_size
heads_xa = 32
dropout = 0.1
emb_dropout = 0.1
max_ar_order = 1
patch_size = 8
max_patches = 4096
max_fields = 3
max_components = 3

# lora settings
rank_lora_attn = 16
rank_lora_mlp = 16
lora_p = 0.05

morph = ViT3DRegression(patch_size=patch_size, dim=dim, depth=depth,
    heads=heads, heads_xa=heads_xa, mlp_dim=mlp_dim,
    max_components=max_components, conv_filter=filters,
    max_ar=max_ar_order,
    max_patches=max_patches, max_fields=max_fields,
    dropout=dropout, emb_dropout=emb_dropout,
    lora_r_attn=rank_lora_attn,            # <— rank of A and B in the attention module
    lora_r_mlp=rank_lora_mlp,              # <— rank of A and B in the MLP module
    lora_alpha=None,                       # defaults to 2*rank inside LoRA
    lora_p=lora_p                          # dropout on LoRA path
).to(device)

# print('Model architecture:', ft_model)
num_params_model = sum(p.numel() for p in morph.parameters()) / 1e6
print(f"→ NUMBER OF PARAMETERS OF THE MODEL (in M): {num_params_model:.3g}")


In [ ]:
# load the foundational model weights
from huggingface_hub import hf_hub_download

# e.g., grab the "Ti" checkpoint (change filename as needed)
weights_path = hf_hub_download(
    repo_id="mahindrautela/MORPH",
    filename="morph-Ti-FM-max_ar1_ep225.pth",
    subfolder="models/FM",
    repo_type="model",              # optional
    resume_download=True,           # continue if interrupted
    local_dir="weights/FM",         # where to place it
    local_dir_use_symlinks=False    # copy file instead of symlink
)
print(weights_path)

In [ ]:
#%% Load weights into the model
import torch.nn as nn

# ---- load the pretrained weights ----
start_epoch = 0
print(f"→ Loading checkpoints from {weights_path}")
# --- Load pretrained checkpoint from foundational model ---
ckpt = torch.load(weights_path, map_location=device, weights_only=True)
state_dict = ckpt["model_state_dict"]

# pick the real model if wrapped   
target = morph.module if isinstance(morph, nn.DataParallel) else morph 

if state_dict and next(iter(state_dict)).startswith("module."):
    print("→ Stripping 'module.' from checkpoints")
    state_dict = {k.replace("module.", "", 1): v for k, v in state_dict.items()}
    
# strict=False because ft_model has extra LoRA params (A/B) not in ckpt
missing, unexpected = target.load_state_dict(state_dict, strict=False)

# sanity print
print("Missing keys (expected: LoRA A/B etc.):",
        [k for k in missing if k.endswith((".A", ".B")) or ".lora" in k])
print("Unexpected keys:", unexpected)
print(f"→ Resumed from {weights_path}, starting at epoch {start_epoch}")


##### Task-specific head (TSH) (Simple CNN1D)

In [ ]:
# Calculate number of patches and features per patch
N_patches_W = X.shape[-2] // patch_size
N_patches_H = X.shape[-1] // patch_size
N_patches = N_patches_W * N_patches_H
feat_per_patch = patch_size * patch_size * X.shape[2] * X.shape[3]
print(f'Number of patches along W: {N_patches_W}, H: {N_patches_H}, Total Patches: {N_patches} \n'
      f'Feat_per_patch (patch_size * patch_size * C * F): {feat_per_patch}')

In [ ]:
# Task-specific head model
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class TaskSpecificHead_CNN1D(nn.Module):
    def __init__(self, n_patches=64, feat_dim=256, output_dim=5, dropout_p=0.1):
        super().__init__()
        self.n_patches = n_patches
        self.feat_dim = feat_dim

        self.conv1 = nn.Conv1d(in_channels=feat_dim, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128,      out_channels=64,  kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64,       out_channels=32,  kernel_size=3, padding=1)

        self.act  = nn.GELU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)  # 64->32->16->8
        self.drop = nn.Dropout(dropout_p)

        feat_len = n_patches // 8                         # 64 -> 8
        self.fc   = nn.Linear(32 * feat_len, output_dim)  # 32*8 = 256

    def forward(self, x):
        B = x.shape[0]
        # reshape to (B, 64, 256) then permute to (B, 256, 64) for Conv1d
        #print(f"[TaskSpecificHead_CNN1D] Input x shape: {x.shape}")
        x = x.view(B, self.n_patches, self.feat_dim).permute(0, 2, 1).contiguous()
        #print(f"[TaskSpecificHead_CNN1D] Reshaped x shape: {x.shape}")
        x = self.pool(self.act(self.conv1(x)))  # (B,128,32)
        x = self.drop(x)
        x = self.pool(self.act(self.conv2(x)))  # (B, 64,16)
        x = self.drop(x)
        x = self.pool(self.act(self.conv3(x)))  # (B, 32, 8)
        x = x.flatten(1)                        # (B, 32*8 = 256)
        x = self.drop(x)
        return self.fc(x)                       # (B, output_dim)
    
# Instantiate the model - 1
output_dim = params.shape[1]  # number of parameters to predict
head = TaskSpecificHead_CNN1D(n_patches = N_patches, feat_dim = dim, output_dim = output_dim).to(device)
print("Num params encoder (in K): ", sum(p.numel()//10**3 for p in head.parameters()))
print('Model architecture', head)

In [ ]:
#%% Fine-tuning setup
from types import SimpleNamespace
from src.utils.select_fine_tuning_parameters import SelectFineTuningParameters

# defining arguments (similar to ft code)
def make_args(l1=False, l2=False, l3=False, l4=False, lr4=1e-4, wd4=0.0):
    return SimpleNamespace(
        ft_level1=l1, ft_level2=l2, ft_level3=l3, ft_level4=l4,
        lr_level4=lr4, wd_level4=wd4
    )

# Level-1 only
args = make_args(l1=True, l2=True)
selector = SelectFineTuningParameters(morph, args)
optimizer_1 = selector.configure_levels()

In [ ]:
# Optimizer for head network
import torch.optim as optim
optimizer_2 = optim.Adam(head.parameters(), lr=1e-3)

In [ ]:
# Loss function
loss_fn = nn.MSELoss()

#### Train/Finetune MORPH + TSH

In [ ]:
# Trainer class
class Trainer:
    @staticmethod
    def train_epoch(dataloader_train, model_1, model_2, optimizer_1, optimizer_2, device):
        model_1.train() 
        model_2.train()
        train_loss_1 = []
        train_loss_2 = []
        for step, batch in enumerate(dataloader_train):
            x_tr, y_tr = batch
            x_tr, y_tr = x_tr.to(device), y_tr.to(device)
            #print(f'X: {x_tr.shape}, y: {y_tr.shape}')
            optimizer_1.zero_grad()
            optimizer_2.zero_grad()

            # Model 1 forward + loss
            _, z, x_nsp = model_1(x_tr)
            #print(f"[Trainer] Model 1 output shape: {x_nsp.shape}")
            #print(f"[Trainer] Model 1 latent shape: {z.shape}")
            loss_1 = loss_fn(x_nsp.unsqueeze(1), x_tr) # autoencoder loss

            # Model 1 backward
            loss_1.backward()
            optimizer_1.step()
            train_loss_1.append(loss_1.item())

            # Model 2 forward + backward
            z = z.detach() # detach the two graphs
            y_hat = model_2(z.squeeze(1))
            #print(f"[Trainer] Model 2 output shape: {y_hat.shape}")
            loss_2 = loss_fn(y_hat, y_tr)
            loss_2.backward()
            optimizer_2.step()
            train_loss_2.append(loss_2.item())

        return np.mean(train_loss_1), np.mean(train_loss_2)

    @staticmethod
    def test_epoch(dataloader_val, model_1, model_2, device):
        model_1.eval() # Set the eval mode for model
        model_2.eval()
        test_loss_1 = []
        test_loss_2 = []
        with torch.no_grad(): # No need to track the gradients
            for step, batch in enumerate(dataloader_val):
                x_val, y_val = batch
                x_val, y_val = x_val.to(device), y_val.to(device)

                # Model 1 forward
                _, z, x_nsp = model_1(x_val)
                loss_1 = loss_fn(x_nsp.unsqueeze(1), x_val)
                test_loss_1.append(loss_1.item())
                
                # Model 2 forward
                y_hat_2 = model_2(z.squeeze(1))
                loss_2 = loss_fn(y_hat_2, y_val)
                test_loss_2.append(loss_2.item())

        return np.mean(test_loss_1), np.mean(test_loss_2)

In [ ]:
import time
diz_loss = {'train_loss_morph':[],'train_loss_head':[], 'val_loss_morph':[], 'val_loss_head':[]}
begin_time = time.time()
for epoch in range(num_epochs):
   train_loss_morph, train_loss_head = Trainer.train_epoch(dataloader_train, morph, head, 
                                                           optimizer_1, optimizer_2, device)
   val_loss_morph, val_loss_head = Trainer.test_epoch(dataloader_val, morph, head, device)

   print(
    f"\n EPOCH {epoch+1}/{num_epochs} TIME: {time.time()-begin_time:.2f}s, "
    f"train loss morph {train_loss_morph:.4f}, "
    f"val loss morph {val_loss_morph:.4f}, "
    f"train loss head {train_loss_head:.4f}, "
    f"val loss head {val_loss_head:.4f}"
)

   # store the losses per epoch
   diz_loss['train_loss_morph'].append(train_loss_morph)
   diz_loss['train_loss_head'].append(train_loss_head)
   diz_loss['val_loss_morph'].append(val_loss_morph)
   diz_loss['val_loss_head'].append(val_loss_head)

# Save the model
torch.save(morph.state_dict(), os.path.join(model_dir, 'morph_ft_icf.pth'))
torch.save(head.state_dict(), os.path.join(model_dir, 'head_ft_icf.pth'))

##### Plot train vs val loss

In [ ]:
# print train and val loss
plt.figure(figsize=(6, 4))
plt.plot(diz_loss['train_loss_morph'], '-ok', label='Train',)
plt.plot(diz_loss['val_loss_morph'], '-^r', label='Valid')
plt.xlabel('Epoch',fontsize=20)
plt.ylabel('Average Loss (MORPH)',fontsize=20)
plt.legend(["tr_total", "val_total"])
plt.title('Training & Validation loss', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()
plt.savefig(os.path.join(results_dir, 'loss_ft_morph_icf.png'))


# print train and val loss
plt.figure(figsize=(6, 4))
plt.plot(diz_loss['train_loss_head'], '-ok', label='Train',)
plt.plot(diz_loss['val_loss_head'], '-^r', label='Valid')
plt.xlabel('Epoch',fontsize=20)
plt.ylabel('Average Loss (HEAD)',fontsize=20)
plt.legend(["tr_total", "val_total"])
plt.title('Training & Validation loss', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()
plt.savefig(os.path.join(results_dir, 'loss_ft_head_icf.png'))

#### Testing MORPH + TSH on test set

In [ ]:
# Testing
def testing(dataloader_test, model_1, model_2, device):
        model_1.eval() # Set the eval mode for model
        model_2.eval()
        mse_loss_1, mse_loss_2 = [], []
        x_org, x_pred, y_org, y_pred = [], [], [], []
        with torch.no_grad(): # No need to track the gradients
            for _, batch in enumerate(dataloader_test):
                x_test, y_test = batch
                x_test, y_test = x_test.to(device), y_test.to(device)

                # Model 1 forward
                _, z, x_nsp = model_1(x_test)
                loss_1 = loss_fn(x_nsp.unsqueeze(1), x_test)
                mse_loss_1.append(loss_1.item())
                
                # Model 2 forward
                y_hat_2 = model_2(z.squeeze(1))
                loss_2 = loss_fn(y_hat_2, y_test)
                mse_loss_2.append(loss_2.item())

                # Collect original and predicted samples
                x_org.append(x_test.cpu().numpy())
                x_pred.append(x_nsp.unsqueeze(1).cpu().numpy())
                y_org.append(y_test.cpu().numpy())
                y_pred.append(y_hat_2.cpu().numpy())

        return mse_loss_1, mse_loss_2, x_org, x_pred, y_org, y_pred

# get the test losses and predictions
mse_loss_main, mse_loss_head, x_org, x_pred, y_org, y_pred = testing(dataloader_test, morph, head, device)
print(f"Test Loss - Morph: {np.mean(mse_loss_main):.4f}, Head: {np.mean(mse_loss_head):.4f}")
print(f'x_org shape: {x_org[0].shape}, x_pred shape: {x_pred[0].shape}')
print(f'y_org shape: {y_org[0].shape}, y_pred shape: {y_pred[0].shape}')

In [ ]:
# plot y_org vs y_pred
def plot_ytrue_vs_ypred(y_org, y_pred):
    y_org_arr = np.concatenate(y_org, axis=0)
    y_pred_arr =  np.concatenate(y_pred, axis=0)
    print(f'y_org shape: {y_org_arr.shape}, y_pred shape: {y_pred_arr.shape}')
    plt.figure(figsize=(32, 4))
    for c in range(y_org_arr.shape[1]):
        r2 = np.corrcoef(y_org_arr[:, c], y_pred_arr[:, c])[0, 1] ** 2
        mse = np.mean((y_org_arr[:, c] - y_pred_arr[:, c])**2)
        print(f'Parameter {c+1}: R^2 = {r2:.4f}, MSE = {mse:.4f}')
        plt.subplot(1, 5, c+1)
        plt.plot(y_org_arr[:, c], y_pred_arr[:, c], 'o', label='Original vs Predicted')
        plt.title(f'Parameter {c+1}, r2 = {r2:.4f}, MSE = {mse:.4f}')
        plt.xlabel('True')
        plt.ylabel('Predicted')
        plt.legend()
    plt.show()
    plt.savefig(os.path.join(HERE, 'ytrue_vs_ypred.png'), dpi=300)

# plot for all parameters
plot_ytrue_vs_ypred(y_org, y_pred)

In [ ]:
# Plots of x_org vs x_pred
def plot_original_vs_predicted(x_org, x_pred):
    len_samples = len(x_org)
    print(f'Number of test samples: {len_samples}, each of shape: {x_org[0].shape}')
    select_samples = np.random.choice(len_samples, 5, replace=False)
    print(f'Selected samples: {select_samples}')

    for idx in select_samples:
        x_o = x_org[idx][:,0,:,:,0,:,:]       # (1, 2, 2, 1, 64, 64) -> (2, 2, 64, 64)
        x_p = x_pred[idx][:,0,:,:,0,:,:]      # (1, 2, 2, 1, 64, 64) -> (2, 2, 64, 64)
        x_o = x_o.reshape(-1, 4, 64, 64)      # (B, 4, 64, 64)
        x_p = x_p.reshape(-1, 4, 64, 64)      # (B, 4, 64, 64)
        print(f'x_org shape: {x_o.shape}, x_pred shape: {x_p.shape}')
        
        idx = np.random.randint(0, x_o.shape[0])  # pick a random sample from the batch
        x_o = x_o[idx]  # (4, 64, 64)
        x_p = x_p[idx]  # (4, 64, 64)

        fig, axes = plt.subplots(2, 4, figsize=(12, 6))
        for c in range(4):
            axes[0, c].imshow(x_o[c, :, :], cmap='plasma', origin='lower')
            axes[0, c].set_title(f'Original - Channel {c}')
            axes[0, c].axis('off')

            axes[1, c].imshow(x_p[c, :, :], cmap='plasma', origin='lower')
            axes[1, c].set_title(f'Predicted - Channel {c}')
            axes[1, c].axis('off')
        plt.suptitle(f'Sample {idx}: Original vs Predicted')
        plt.tight_layout()
        plt.show()
        plt.savefig(os.path.join(HERE, f'original_vs_predicted_sample_{idx}.png'), dpi=300)

plot_original_vs_predicted(x_org, x_pred)

## Standalone Model on ICF data

#### Standalone surrogate (SS): CNN2D encoder for regression

In [ ]:
# Standalone model
import torch.nn as nn

class Standalone_CNN2D(nn.Module):
    def __init__(self, output_dim=5):
        super(Standalone_CNN2D, self).__init__()
        # 64x64 input with 4 channels
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=32, kernel_size=3, stride=1, padding=1)   # -> 32 x 64 x 64
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)  # -> 64 x 32 x 32
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1) # -> 128 x 16 x 16
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1) # -> 256 x 8 x 8

        self.activation = nn.GELU()
        self.dropout = nn.Dropout(0.1)
        self.flatten = nn.Flatten()

        # After three 2x2 pools: 64 -> 32 -> 16 -> 8
        # So features = 256 * 8 * 8 = 16384
        self.fc1 = nn.Linear(256 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = self.activation(self.conv1(x))  # 32 x 64 x 64
        x = self.activation(self.conv2(x))  # 64 x 32 x 32
        x = self.activation(self.conv3(x))  # 128 x 16 x 16
        x = self.activation(self.conv4(x))  # 256 x 8 x 8
        x = self.flatten(x)                  # flatten
        x = self.activation(self.fc1(x))    # 256
        x = self.dropout(x)
        return self.fc2(x)                              # logits of shape (B, output_dim)

# Instantiate the model - 2
output_dim = params.shape[1]  # number of parameters to predict
model_ss = Standalone_CNN2D(output_dim).to(device)
print("Num params encoder (in M): ", sum(p.numel()//10**6 for p in model_ss.parameters()))
print('Model architecture', model_ss)

#### Training SS (CNN2D)

In [ ]:
# Trainer class
class Trainer2:
    @staticmethod
    def train_epoch_ss(dataloader_train, model, optimizer, device):
        model.train() 
        train_loss = []
        for step, batch in enumerate(dataloader_train):
            x_tr, y_tr = batch
            x_tr, y_tr = x_tr.to(device), y_tr.to(device)
            # print(f'X: {x_tr.shape}, y: {y_tr.shape}')
            # reshape x_tr
            x_tr = x_tr[:, 0, :, :, 0, :, :].reshape(-1, 4, 64, 64)  # (B,T,F,C,D,H,W) -> (B, 4, 64, 64)
            # print(f'X: {x_tr.shape}, y: {y_tr.shape}')

            optimizer.zero_grad()

            # Model 1 forward + loss
            y_hat = model(x_tr)
            loss = loss_fn(y_hat, y_tr) 

            # Model 1 backward
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

        return np.mean(train_loss)

    @staticmethod
    def test_epoch_ss(dataloader_val, model, device):
        model.eval() # Set the eval mode for model
        test_loss = []
        with torch.no_grad(): # No need to track the gradients
            for step, batch in enumerate(dataloader_val):
                x_val, y_val = batch
                x_val, y_val = x_val.to(device), y_val.to(device)

                # print(f'X: {x_tr.shape}, y: {y_tr.shape}')
                # reshape x_tr
                x_val = x_val[:, 0, :, :, 0, :, :].reshape(-1, 4, 64, 64)  # (B,T,F,C,D,H,W) -> (B, 4, 64, 64)
                # print(f'X: {x_val.shape}, y: {y_val.shape}')

                # Modelforward
                y_hat = model(x_val)
                loss = loss_fn(y_hat, y_val)
                test_loss.append(loss.item())

        return np.mean(test_loss)

In [ ]:
import time
diz_loss_ss = {'train_loss':[], 'val_loss':[]}
begin_time = time.time()
for epoch in range(num_epochs):
   train_loss = Trainer2.train_epoch_ss(dataloader_train, model_ss, optimizer_2, device)
   val_loss = Trainer2.test_epoch_ss(dataloader_val, model_ss, device)

   print(
    f"\n EPOCH {epoch+1}/{num_epochs} TIME: {time.time()-begin_time:.2f}s, "
    f"train loss {train_loss:.4f}, "
    f"val loss {val_loss:.4f}"
)

   # store the losses per epoch
   diz_loss_ss['train_loss'].append(train_loss)
   diz_loss_ss['val_loss'].append(val_loss)

# Save the model
torch.save(model_ss.state_dict(), os.path.join(model_dir, 'model_ss_icf.pth'))

##### plot training and val loss for SS

In [ ]:
# print train and val loss
plt.figure(figsize=(10, 6))
plt.plot(diz_loss_ss['train_loss'], '-ok', label='Train',)
plt.plot(diz_loss_ss['val_loss'], '-^r', label='Valid')
plt.xlabel('Epoch',fontsize=20)
plt.ylabel('Average Loss',fontsize=20)
plt.legend(["tr_total", "val_total"])
plt.title('Training & Validation loss (Standalone model)', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()
plt.savefig(os.path.join(results_dir, 'loss_ss_icf.png'))

#### Testing SS on test set

In [ ]:
# Testing
def testing_ss(dataloader_test, model, device):
        model.eval() # Set the eval mode for model
        test_loss = []
        y_org, y_pred = [], []
        with torch.no_grad(): # No need to track the gradients
            for _, batch in enumerate(dataloader_test):
                x_test, y_test = batch
                x_test, y_test = x_test.to(device), y_test.to(device)

                # print(f'X: {x_tr.shape}, y: {y_tr.shape}')
                # reshape x_tr
                x_test = x_test[:, 0, :, :, 0, :, :].reshape(-1, 4, 64, 64)  # (B,T,F,C,D,H,W) -> (B, 4, 64, 64)
                # print(f'X: {x_test.shape}, y: {y_test.shape}')

                # Model forward
                y_hat = model(x_test)
                loss = loss_fn(y_hat, y_test)
                test_loss.append(loss.item())

                # Collect original and predicted samples
                y_org.append(y_test.cpu().numpy())
                y_pred.append(y_hat.cpu().numpy())

        return test_loss, y_org, y_pred

# get the test losses and predictions
test_losses_ss, y_org_ss, y_pred_ss = testing_ss(dataloader_test, model_ss, device)
print(f'y_org shape: {y_org_ss[0].shape}, y_pred shape: {y_pred_ss[0].shape}')

In [ ]:
# plot for all parameters
plot_ytrue_vs_ypred(y_org_ss, y_pred_ss)